In [5]:
#!/Users/florian/anaconda3/bin/python
import bs4
import requests
import genanki
import sys
import PyPDF2
import pandas as pd
import pytesseract
from PIL import Image
from googletrans import Translator
import re


In [13]:
#Function to translate to english
def translate(words):
    try:
        translator = Translator()
        translate_list = []
        trans = translator.translate(words, dest = 'en') 
        for translation in trans:
            translate_list.append(translation.text)
        return translate_list
    except:
        return None


In [15]:
# Open the image file
image = Image.open('test.jpg')
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'
# Perform OCR using PyTesseract
text = pytesseract.image_to_string(image, lang='deu')
pattern = r'\d+|/|\.b|\.a|\.c|\.|Start|der|die|das'
text = re.sub(pattern, '', text)
# Text uprava
text_list = text.splitlines()
#zbavit se prazdnych stringu
text_ohneempty = [i for i in text_list if len(i)>=2]
#zbavit se stringu s cisly

text_final = pd.DataFrame(text_ohneempty)
text_final.columns = ['German']

# Print the extracted text
# print(text_final)

text_final['English'] = translate(text_final['German'].tolist())
print(text_final)


                            German                              English
0                      Wörterliste                            Word list
1            Großvater,  Großväter            Grandfather, grandfathers
2                   Grund,  Gründe                      Reason, reasons
3             gründen, er gründet,                   found, he founded,
4                 er hat gegründet                           he founded
..                             ...                                  ...
101   Intensivkurs,  Intensivkurse  Intensive course, intensive courses
102                             nn                                   nn
103                            u |                                  u |
104                             la                                  the
105                              b                                    b

[106 rows x 2 columns]


In [19]:
#display(text_final)
items = []

for index, row in text_final.iterrows():
    german = row['German']
    english = row['English']
    items.append((german, english))

my_model = genanki.Model(
  1607392319,
  'Simple Model',
  fields=[
    {'name': 'german'},
    {'name': 'english'},
  ],
  templates=[
      {
          'name': 'Forward Card',
          'qfmt': '{{german}}',
          'afmt': '{{FrontSide}}<hr id="answer">{{english}}',
      },
      {
          'name': 'Reverse Card',
          'qfmt': '{{english}}',
          'afmt': '{{FrontSide}}<hr>{{german}}'
      }
  ])

my_deck = genanki.Deck(2059400110, "MH Booklet 1")

for i in items:
    my_deck.add_note(genanki.Note(
        model=my_model,
        fields=i))

genanki.Package(my_deck).write_to_file('output.apkg')
    

TypeError: expected string or bytes-like object, got 'NoneType'